# Class 6: The Bigger Picture

## Guest speakers

### [Elizabeth Marcello, PhD](https://www.linkedin.com/in/elizabeth-marcello-ph-d-686919b0/)

> Elizabeth is a Senior Research Analyst at [Reinvent Albany](https://reinventalbany.org/). She cut her teeth in New York State policy and politics at Reinvent Albany following several years of working in Senegal and then Kenya. After a hiatus to finish her PhD, Elizabeth returned to Reinvent Albany to work on business subsidy and economic development issues. Elizabeth has a master’s degree from Columbia University’s School of International and Public Affairs and a PhD in urban planning from Columbia's Graduate School of Architecture, Planning and Preservation.

#### Questions?

### [Joel Natividad](https://www.linkedin.com/in/joelnatividad/)

> A social entrepreneur working on data portals & data infrastructure since 2010, Joel has helped deploy more than 50 open source, open data solutions at all levels of government across the US. After winning the NYCBigApps Challenge twice, Joel and his cofounder started Ontodia - becoming the first CKAN professional services partner in the Americas in 2013 – deploying data portals in Jersey City, Newark, Allegheny County, PA; Boston, San Antonio, NYC Department of Education, and UNDP among others; launching CivicDashboards at Code for America 2015, before being acquired by OpenGov in 2016. In 2019, they left OpenGov and started datHere.

#### Full version

> A social entrepreneur working on data portals & data infrastructure since 2010, Joel has helped deploy more than 50 open source, open data solutions at all levels of government across the US.
>
> The son of two civil servants who always wanted to apply his technology skills in the public sector, he finally got his chance to do so when he convinced his boss to enter the [NYCBigApps Challenge](https://en.wikipedia.org/wiki/NYC_BigApps) in 2011 - where their entry won the "Large Organization" Award.
>
> When the 2012 NYCBigApps Challenge was announced however, his boss declined to participate again, citing that the anticipated business from winning did not materialize as expected.
>
> Still, he decided to go for it and turn in his papers, working out of Sami - his co-founder's basement, for the next six months. Their gamble paid off when they won the NYCBigApps Grand Prize in 2012.
>
> Sami and Joel decided to apply themselves fulltime and started their first civictech startup Ontodia - becoming the first [CKAN](https://ckan.org/) professional services partner in the Americas in 2013 – deploying data portals in Jersey City, Newark, Allegheny County, PA; Boston, San Antonio, NYC Department of Education, and UNDP among others; launching CivicDashboards at Code for America 2015, before being acquired by [OpenGov](https://opengov.com/) in 2016.
>
> At OpenGov, they built the largest CKAN SaaS service in the Americas, while actively contributing/sponsoring core CKAN development and staying active in the community.
>
> However, they found that OpenGov's SaaS business model constricting - as it disincentivized "building with, not for" their public sector customers, so they left OpenGov and started [datHere](https://dathere.com/) in 2019.
>
> With datHere – they apply themselves anew to the data management challenge – building open source, data management infrastructure to ensure that your Data is Useful, Usable and Used.

#### Questions?

## Final Project wrap-up

- Heard from lots of people having issues running out of disk, kernel crashing, 500 errors when exporting, etc.
   - Can only [anticipate](https://python-public-policy.afeld.me/en/{{school_slug}}/assignments.html#common-issues) so much
- [Warning from first day](https://python-public-policy.afeld.me/en/{{school_slug}}/lecture_0.html#you)
- This is (unfortunately) a big part of coding, and thus the Final Project
- Troubleshooting is a skill
   - "It's broken"

## Ask Me Anything (AMA)

Have slides on "Python beyond data analysis" as backup, but would rather talk about what you want to hear about.

## Python beyond data analysis

We've been focusing on using Python and pandas for data analysis. What else is Python used for?

### Data engineering

- Automation / recurring processes
- Copying/moving/processing/publishing data, especially Big Data
- Monitoring/alerting

### Web development

- Building web sites that are interactive (more than just content)
- Forms
- Presenting data
- Workflows, such as:
   - Signing up for things
   - Paying for things

### Machine learning

- Statistics, but _fancy_
- Building models
   - [Examples](https://tfhub.dev/)
- Finding patterns
- Recommendations
- Detection

When people say "artificial intelligence," they usually mean "machine learning."

![Diagram showing what type of machine learning may be useful, if at all](https://mitsloan.mit.edu/sites/default/files/2021-04/machine-learning-infographic_2.jpg)

[Source, with more thorough explanation](https://mitsloan.mit.edu/ideas-made-to-matter/machine-learning-explained)

#### The process

_High-level_

1. Create a model
   1. Gather a bunch of data for training
   1. If supervised machine learning, label it (give it the right answers)
   1. Segment into training and test data
   1. Train the model against the training dataset (have it identify patterns)
   1. Test the model against the test dataset
1. Run against new data
1. If reinforcement learning, model refines itself

You have a head start: [The fundamentals](https://python-public-policy.afeld.me/en/{{school_slug}}/syllabus.html#learning-objectives) are applicable anywhere you're using code.

## Post-{{coding_env_name}}

- {{coding_env_name}} instance (and your copy of files) will be deleted
    - Download things you want to keep, particularly edited notebooks
- Class materials will remain available on [the site](https://python-public-policy.afeld.me/en/{{school_slug}}/) and [through GitHub](https://github.com/afeld/python-public-policy/tree/{{school_slug}})
- [Running notebooks elsewhere](https://python-public-policy.afeld.me/en/{{school_slug}}/resources.html#jupyter-outside-this-course)

Thanks to the {{assistant_name}}!

## Thank you!

Keep in touch:

- [Email](https://python-public-policy.afeld.me/en/{{school_slug}}/syllabus.html#instructor-information)
- [@aidanfeldman](https://twitter.com/aidanfeldman/) on Twitter